## Part 2: Embedding Julia into Python

We can use PythonCall for integrating Python’s vast ecosystem into Julia projects and JuliaCall for embedding high-performance Julia code into Python scripts.

You’ll see how easy it is to blend these languages and why it’s worth the effort.

In [ ]:
from juliacall import Main as jl

In [ ]:
%load_ext juliacall

In [ ]:
%julia using Pkg

In [ ]:
%julia Pkg.add("UnROOT")

In [ ]:
%julia using UnROOT

In [ ]:
file = jl.Main.ROOTFile("./data/SMHiggsToZZTo4L.root")

In [ ]:
%%timeit
jl.Main.ROOTFile("./data/SMHiggsToZZTo4L.root")

In [ ]:
file

In [ ]:
events = jl.Main.LazyTree(file, "Events")

In [ ]:
%%timeit
jl.Main.LazyTree(file, "Events")

In [ ]:
jl.include('awkward_analyzer_functions.jl')

```julia
using AwkwardArray

function make_record_array(events)
    array = AwkwardArray.RecordArray(
        NamedTuple{(:pt, :eta, :phi, :mass, :charge, :isolation)}((
            AwkwardArray.from_iter(events.Muon_pt),
            AwkwardArray.from_iter(events.Muon_eta), 
            AwkwardArray.from_iter(events.Muon_phi), 
            AwkwardArray.from_iter(events.Muon_mass), 
            AwkwardArray.from_iter(events.Muon_charge), 
            AwkwardArray.from_iter(events.Muon_pfRelIso03_all),
        )
    ))
    return AwkwardArray.convert(array)
end
```

In [ ]:
muons = jl.make_record_array(events)

In [ ]:
%%time
jl.make_record_array(events)

In [ ]:
muons.show(type=True)

In [ ]:
import awkward as ak

In [ ]:
muons = ak.zip({
                "pt": muons.pt,
                "eta": muons.eta,
                "phi": muons.phi,
                "mass": muons.mass,
                "charge": muons.charge,
                "isolation": muons.isolation,
            },
            with_name="PtEtaPhiMCandidate",)

In [ ]:
cutflow = dict()

# Sort muons by transverse momentum
muons = muons[ak.argsort(muons.pt, axis=1)]

cutflow["all events"] = ak.num(muons, axis=0)

# Quality and minimum pt cuts
muons = muons[(muons.pt > 5) & (muons.isolation < 0.2)]
cutflow["at least 4 good muons"] = ak.sum(ak.num(muons) >= 4)

In [ ]:
# reduce first axis: skip events without enough muons
muons = muons[ak.num(muons) >= 4]

```julia
four_muons = AwkwardArray.ListArray{
    AwkwardArray.Index64,
    AwkwardArray.ListArray{
        AwkwardArray.Index64,
        AwkwardArray.PrimitiveArray{Int64},
    },
}()

function find_4lep(events_leptons)

    for leptons in events_leptons
        nlep = length(leptons)
        for i0 in 1:nlep
            for i1 in (i0 + 1):nlep
                if leptons[i0][:charge] + leptons[i1][:charge] != 0
                    continue
                end
                for i2 in 1:nlep
                    for i3 in (i2 + 1):nlep
                        if length(Set([i0, i1, i2, i3])) < 4
                            continue
                        end
                        if leptons[i2][:charge] + leptons[i3][:charge] != 0
                            continue
                        end
                        
                        push!(four_muons.content.content, (i0 - 1)) # Julia is 1-based, subtract 1 for 0-based indexing
                        push!(four_muons.content.content, (i1 - 1))
                        push!(four_muons.content.content, (i2 - 1))
                        push!(four_muons.content.content, (i3 - 1))
                        AwkwardArray.end_list!(four_muons.content)
                    end
                end
            end 
        end
        AwkwardArray.end_list!(four_muons)
    end
    return four_muons
end

```

In [ ]:
good_four_muons = jl.find_4lep(muons[1:10])

In [ ]:
good_four_muons = jl.find_4lep(muons)

In [ ]:
%%time
jl.find_4lep(muons)

In [ ]:
fourmuon = jl.AwkwardArray.convert(good_four_muons)

In [ ]:
fourmuon.show(type=True)

Let's go to the next [notebook](AwkwardArray_Julia_Python-part3.ipynb).